In [14]:
import sys
import sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
import csv

In [44]:
# Read file
filename = "30000-40000.csv"
file = open("../data/30000-40000.csv")
reader = csv.reader(file)
i = 0
y = []
X = []
names = []
temp = next(reader) # skip label row
for row in reader:
    if len(row) < 6:
        continue
    name = row[0]
    location = row[1]
    score = row[2]
    reviews = row[3]
    salaries = row[4]
    interviews = row[5]
    
    names.append(name)
    
    # y labels
    y.append(score)
    
    # X features
    feature = []
#     feature.append(location)
    feature.append(reviews)
    feature.append(salaries)
    feature.append(interviews)
    
    X.append(feature)
    
    i = i + 1

In [45]:
# Convert to numpy array
X = np.array(X)
y = np.array(y)

In [43]:
model = LinearRegression()
model.fit(X, y)

ValueError: could not convert string to float: '1.3k'